In [1]:
# My Google Drive Mount하기!
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [23]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

print(len(train_data))
print(len(test_data))

5
5


In [24]:
print(train_data.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [25]:
print(train_data.target[0])

7


* target label 목록 확인하기

In [26]:
print(len(train_data.target_names))
print(train_data.target_names)

20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


# TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tdm = TfidfVectorizer(
    max_features=10000,
    stop_words='english'
)

train_input = tdm.fit_transform(train_data.data)
test_input = tdm.transform(test_data.data)

# One Hot Encoding
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

train_target_onehot = lb.fit_transform(train_data.target)
test_target_onehot = lb.transform(test_data.target)

print(train_input.shape, train_target_onehot.shape)
print(test_input.shape, test_target_onehot.shape)

(11314, 10000) (11314, 20)
(7532, 10000) (7532, 20)


In [28]:
print(train_data.target[:5])
print(train_target_onehot[:5])

[ 7  4  4  1 14]
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]]


In [29]:
print(test_data.target[:5])
print(test_target_onehot[:5])

[ 7  5  0 17 19]
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


# Save the data

In [30]:
import joblib

FolderPath = '/content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data'

joblib.dump(
    {
        'tdm': tdm,
        'TargetName_list': train_data.target_names,
        'train_input': train_input,
        'train_target_onehot': train_target_onehot,
        'test_input': test_input,
        'test_target_onehot': test_target_onehot
    },
    FolderPath + '/' + 'newsgroup.pkl'
)

['/content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data/newsgroup.pkl']

# Classification + Dropout + BatchNormalization Practice

In [31]:
import joblib

data = joblib.load(FolderPath + '/' + 'newsgroup.pkl')
locals().update(data)

## (1) Single Layer Model

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(20,
                activation='softmax',
                input_shape=(train_input.shape[-1], )))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                200020    
Total params: 200,020
Trainable params: 200,020
Non-trainable params: 0
_________________________________________________________________


In [33]:
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.01),
    metrics=['acc']
)

In [34]:
from tensorflow.keras.callbacks import EarlyStopping

model.fit(
    train_input.A,
    train_target_onehot,
    epochs=30,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_acc')]
)

Epoch 1/30
319/319 [==============================] - 1s 3ms/step - loss: 1.6757 - acc: 0.7876 - val_loss: 0.9227 - val_acc: 0.8940


* Weights 확인하기

In [35]:
import pandas as pd

w, b = model.weights

weight_df = pd.DataFrame(
    w.numpy(),
    columns=TargetName_list,
    index=tdm.get_feature_names()
)

print(weight_df.shape)

(10000, 20)


In [36]:
scispace_df = weight_df.sort_values('sci.space', ascending=False)[['sci.space']]
scispace_df.head()

,sci.space
space,1.464302
nasa,1.132023
launch,1.006097
moon,1.001863
sci,0.978228


## (2) Multiple Layer Model

In [37]:
from tensorflow.keras.layers import BatchNormalization, Activation

model = Sequential()

model.add(Dense(64, input_shape=(train_input.shape[-1], )))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(20, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                640064    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 20)               

* y를 One Hot Encoding 하기 전으로 돌리자. (sparse categorical crossentropy하기 위함임)

In [39]:
import numpy as np

# Vector(0, 1, 2, ..., 19)를 곱하면, 1이 있는 위치의 값만 남는다!
train_target = train_target_onehot @ np.arange(20)
test_target = test_target_onehot @ np.arange(20)

# 혹은 가장 큰 값의 위치를 찾아도 된다.
# train_target = np.argmax(train_target, axis=1)

print(train_target[:5])
print(test_target[:5])

[ 7  4  4  1 14]
[ 7  5  0 17 19]


In [40]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.01),
    metrics=['acc']
)

model.fit(
    train_input.A,
    train_target,
    epochs=30,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_acc')]
)

Epoch 1/30
319/319 [==============================] - 2s 5ms/step - loss: 0.8145 - acc: 0.7671 - val_loss: 0.4475 - val_acc: 0.8648


## (3) Label Smoothing

* 방법 1: 직접 계산하기

In [41]:
e = 0.2

train_target_smooth = train_target_onehot * (1 - e) + (e / 20)
test_target_smooth = test_target_onehot * (1 - e) + (e / 20)

print(train_target_smooth[:5])
print(test_target_smooth[:5])

[[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.81 0.01 0.01 0.01 0.01 0.01]]
[[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.01 0.81 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.81 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.01 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01 0.01 0.81 0.01 0.01]
 [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.

* 방법 2: label_smoothing 설정하기

In [42]:
from tensorflow.keras.losses import CategoricalCrossentropy

model.compile(
    loss=CategoricalCrossentropy(label_smoothing=0.2),
    optimizer=Adam(learning_rate=0.01),
    metrics=['acc']
)

# Sparse Categorical Crossentropy

* y가 One Hot Encoding된 경우

In [43]:
model.compile(
    loss='categorical_crossentropy'
)

* y가 One Hot Encoding 안된 경우

In [44]:
model.compile(
    loss='sparse_categorical_crossentropy'
)